In [1]:
import math
import collections
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
import utils

In [25]:
MonthlyStatus = collections.namedtuple('MonthlyStatus',
                                       ['mortgage_amount', 'price_index', 'current_price'])


def get_price_index(year_growth_rate, month):
    return math.pow(1 + year_growth_rate/12, month)


class StockPurchase:
    def __init__(self, amount, purchase_price):
        self.amount = amount
        self.purchase_price = purchase_price
        self.current_price = purchase_price
        self.units = self.amount / self.purchase_price
        self.value = self.units * self.current_price
    
    def update_value(self, current_price):
        self.current_price = current_price
        self.value = self.units * self.current_price
        return self
    
    def to_dict(self):
        return {'amount': self.amount,
                'purchase_price': self.purchase_price,
                'current_price': self.current_price,
                'value': self.value}


class Portfolio:
    def __init__(self):
        self.purchases = []
    
    def purchase(self, amount, purchase_price):
        self.purchases.append(StockPurchase(amount, purchase_price))
        return self
    
    def update_values(self, current_price):
        for purchase in self.purchases:
            purchase.update_value(current_price)
        return self


class Simulation:
    def __init__(self, scenario, purchase_price, mortgage):
        self.growth_rate_real_estate = scenario['growth_rate_real_estate']
        self.growth_rate_stocks = scenario['growth_rate_stocks']
        self.currency_exchange_rate = scenario['currency_exchange_rate']
        self.overpayment_amount = scenario['mortgage_overpay_month']
        self.purchase_price = purchase_price
        self.mortgage = mortgage
        self.mortgage_history = []
        self.payment_history = []
    
    def get_monthly_status(self, month):
        mortgage_amount = self.mortgage.mortgage_amount
        price_index = get_price_index(self.growth_rate_real_estate, month)
        current_price = (self.purchase_price + (self.purchase_price * (price_index - 1)))
        return MonthlyStatus(mortgage_amount, price_index, current_price)

    def run(self):
        i = 0
        max_periods = self.mortgage.maturity * self.mortgage.n_periods
        new_monthly_payment_amount = self.mortgage.monthly_payment + self.overpayment_amount
        self.mortgage.update_monthly_payment_amount(new_monthly_payment_amount)
        while self.mortgage.mortgage_amount > 0 and i <= max_periods:
            self.mortgage_history.append(self.get_monthly_status(i))
            self.payment_history.append(self.mortgage.get_next_payment())
            i += 1

In [30]:
portf = Portfolio()
init_price = 100
year_growth_rate = 0.05
for i in range(12):
    curr_price = get_price_index(year_growth_rate, i) * init_price
    portf.purchase(5000, curr_price)
    portf.update_values(curr_price)

In [31]:
pd.DataFrame.from_records([el.to_dict() for el in portf.purchases])

,amount,purchase_price,current_price,value
0,5000,100.000000,104.680023,5234.001151
1,5000,100.416667,104.680023,5212.283304
2,5000,100.835069,104.680023,5190.655573
3,5000,101.255216,104.680023,5169.117583
4,5000,101.677112,104.680023,5147.668962
5,5000,102.100767,104.680023,5126.309340
6,5000,102.526187,104.680023,5105.038347
7,5000,102.953379,104.680023,5083.855615
8,5000,103.382352,104.680023,5062.760778
9,5000,103.813111,104.680023,5041.753472


In [3]:
scenario_1 = {'growth_rate_real_estate': 0.05,
              'growth_rate_stocks': 0.05,
              'currency_exchange_rate': 8.5,
              'mortgage_overpay_month': 5000}

mort = utils.Mortgage(50, 0.0305, 3.6e6, 30, 12)
sim = Simulation(scenario_1, 4.2e6, mort)

In [4]:
sim.run()

In [5]:
result = pd.DataFrame(sim.payment_history).join(pd.DataFrame(sim.mortgage_history))

In [6]:
result['monthly_payment'] = result.interest_amount + result.capital_downpayment_amount + result.fee

In [7]:
result.head()

,fee,interest_amount,capital_downpayment_amount,mortgage_amount,price_index,current_price,monthly_payment
0,50,9150.000000,11124.995854,3.600000e+06,1.000000,4.200000e+06,20324.995854
1,50,9121.723969,11153.271885,3.588875e+06,1.004167,4.217500e+06,20324.995854
2,50,9093.376069,11181.619785,3.577722e+06,1.008351,4.235073e+06,20324.995854
3,50,9064.956119,11210.039735,3.566540e+06,1.012552,4.252719e+06,20324.995854
4,50,9036.463935,11238.531919,3.555330e+06,1.016771,4.270439e+06,20324.995854


In [8]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 237 entries, 0 to 236
Data columns (total 7 columns):
fee                           237 non-null int64
interest_amount               237 non-null float64
capital_downpayment_amount    237 non-null float64
mortgage_amount               237 non-null float64
price_index                   237 non-null float64
current_price                 237 non-null float64
monthly_payment               237 non-null float64
dtypes: float64(6), int64(1)
memory usage: 13.1 KB


In [9]:
point_in_time = 12
paid_interests = result.interest_amount.iloc[:point_in_time].sum() + result.fee.iloc[:point_in_time].sum()
profit = result.current_price.iloc[point_in_time] - result.current_price.iloc[0]
equity = result.capital_downpayment_amount.iloc[:point_in_time].sum()
deposit = result.current_price.iloc[0] - result.mortgage_amount[0]
print('Paid interests to date: {:.0f}'.format(paid_interests))
print('Profit in case of sale: {:.0f}'.format(profit))
print('Equity to date: {:.0f}'.format(equity))

Paid interests to date: 108518
Profit in case of sale: 214880
Equity to date: 135382


In [10]:
total_revenue = equity + profit + deposit
print('Total revenue: {:.0f}'.format(total_revenue))

Total revenue: 950262
